In [ ]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import plotly.graph_objs as go
import plotly.plotly as py
import datetime as dt
import matplotlib.dates as mdates


In [ ]:
data = pd.read_csv("../input/crypto-markets.csv")

In [ ]:
data.head()

In [ ]:
#set up a column with an index so I can only sample 1/10th of the dataset 
data["row_id"]= range(1, len(data) + 1)

In [ ]:
data.head()

In [ ]:
#drop columns I'm not using
data_=data.drop(['slug','ranknow','volume','market'], axis=1)
data_.head()

In [ ]:
data_.set_index('row_id', inplace=True)
data_.head()

In [ ]:
#set date to timestamp format
data_['date'] = pd.to_datetime(data_['date'], format='%Y-%m-%d')

In [ ]:
#pick out the currency for two years span (2016>)
date = data_[data_['date'] >= dt.date(2016, 1, 1)]

In [ ]:
#show if each one closed up or down each day
date['pos_neg']= date['open']-date['close']
date.head()
#create a binary column - 0 = gain, 1 = loss to have something to predict
date['Up/Down'] = np.where(date['pos_neg']>0, '0', '1')

In [ ]:
#create data sets for six crypto currencies
ltcdate = date[date['symbol']=='LTC']
zcashdate = date[date['symbol']=='ZEC']
rippledate = date[date['symbol']=='XRP']
etherdate = date[date['symbol']=='ETH']
mondate = date[date['symbol']=='XMR']
bitdate = date[date['symbol']=='BTC']

In [ ]:
#date.head()

In [ ]:
zcashdate.head()

In [ ]:
#concat the six frames into one
frames=[bitdate,ltcdate,rippledate,etherdate,mondate,zcashdate]
top_six = pd.concat(frames)

In [ ]:
top_six.info()

In [ ]:
#this chart shows each of the six and the number of times they closed up for the day.  
sns.set_style('whitegrid')
sns.countplot(x='symbol',hue='Up/Down',data=top_six,palette='rainbow')

In [ ]:
#a comparison the the up/down ratio at the end of the day for the six.
sns.set(style="whitegrid")

g = sns.factorplot("symbol", "close_ratio", "Up/Down",
                    data=top_six, kind="bar",
                    size=6, palette="muted",
                   legend_out=True)
g.despine(left=True)
g.set_ylabels("Ratio Count")

In [ ]:
#This chart above suggests that even though the days with gains outnumbered (in most cases) the losses, the loss ratios
#tended to be about double the gain ratios. 

In [ ]:
sixtest=top_six.drop('date',axis=1).drop('symbol',axis=1).drop('name',axis=1)
sixtest.head()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = sixtest[['open', 'high', 'low','close', 'close_ratio','spread']]
y = sixtest['Up/Down']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.30, 
                                                    random_state=101)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)

In [ ]:
predictions = logmodel.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test,predictions))